In [ ]:
import torch
from torch import nn

# Model

In [ ]:
class EnergyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(256, 1)
        return None
    def energy(self, x):
        return self.layer(x)
    def forward(self, x: torch.Tensor):
        x.requires_grad_(True)
        energy = self.energy(x)
        y = energy.sum()
        grad = torch.autograd.grad([y], [x], create_graph=False)[0] # for inference only
        if not grad is None: # grad might be None for the JIT compiler
            grad = - grad
        return energy, grad
model = EnergyModel()
input = torch.randn((4, 256))
model = torch.jit.script(model)
model(input)

In [ ]:
model = EnergyModel()
input = torch.randn((4, 256))
model = torch.jit.script(model)
model(input)

# Scripting Example
https://github.com/pytorch/pytorch/issues/46483

In [ ]:
import torch

x = torch.randn(16, 30, 10, requires_grad=True).cuda()
y = torch.randn(16, 10, 1, requires_grad=True).cuda()
z = torch.randn(16, 30, 1, requires_grad=True).cuda()


@torch.jit.script
def sample_function(x: torch.Tensor, y:torch.Tensor, z:torch.Tensor):
    xy = torch.bmm(x, y)
    diff = z - xy
    grad = torch.autograd.grad([diff.mean()], [y])[0]
    if grad is not None:
        y = y - grad
    return y